In [2]:
import pyautogui as py
import cv2
import numpy as np

In [3]:
def takeScreenshot():
    im = py.screenshot("Screen.png", region=(272,131,1023,577))
    img = np.array(im)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

In [4]:
def canny(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = 5
    blur = cv2.GaussianBlur(gray, (kernel,kernel), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

In [5]:
def region_of_interest(img):
    height = img.shape[0]
    width = img.shape[1]
    mask = np.zeros_like(img)
    triangle = np.array([[(0,height),(width/2,height/2-70),(width,height-60),]],np.int32)
    cv2.fillPoly(mask,triangle, 255)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [6]:
def houghLines(img):
    houghLines = cv2.HoughLinesP(img, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap = 10)
    return houghLines

In [13]:
def make_points(img, lineSI):
    slope, intercept = lineSI
    height = img.shape[0]
    y1 = int(height)
    y2 = int(y1*3.0/5)
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return [[x1,y1, x2,y2]]

In [8]:
def average_slope_intercept(img, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        for x1,y1, x2, y2 in line:
            fit = np.polyfit((x1,x2),(y1,y2),1)
            slope = fit[0]
            intercept = fit[1]

            if slope < 0:
                left_fit.append((slope, intercept))
            else:
                right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis = 0)
    right_fit_average = np.average(right_fit, axis = 0)
    left_line = make_points(img, left_fit_average)
    right_line = make_points(img, right_fit_average)
    average_lines = [left_line, right_line]
    return average_lines

In [15]:
def display_lines_average(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for lines in lines:
            for x1,y1,x2,y2 in lines:
                cv2.line(img, (x1,y1),(x2,y2),(0,0,255),10)
    return img

frame = takeScreenshot()
canny_image = canny(frame)
m_img = region_of_interest((canny_image))
h_lines = houghLines(m_img)
avg_lines = average_slope_intercept(frame, h_lines)
print(avg_lines)
lines_image = display_lines_average(frame, avg_lines)
cv2.startWindowThread()
cv2.namedWindow("win")
cv2.imshow("win", lines_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[-14, 577, 443, 346]], [[1141, 577, 555, 346]]]


frame = takeScreenshot()
canny_image = canny(frame)
m_img = region_of_interest((canny_image))
h_lines = houghLines(m_img)
avg_lines = average_slope_intercept(frame, h_lines)
print(avg_lines)
cv2.startWindowThread()
cv2.namedWindow("win")
cv2.imshow("win", m_img)
cv2.waitKey(0)
cv2.destroyAllWindows()